In [1]:
import pandas as pd

df = pd.DataFrame({
    "acctNo": ["123", "123", "123", "456", "789", "789"],
    "story": ["connected_persons", "connected_persons", "board_resolutions", 
              "connected_persons", "connected_persons", "board_resolutions"],
    "message": ["message A", "message B", "message C", "message A", "message B", "message C"],
    "risk": ["low risk", "low risk", "high risk", "low risk", "low risk", "high risk"],
})
df.story = df.story.astype("category")
df.risk = df.risk.astype("category")

In [2]:
df

,acctNo,story,message,risk
0,123,connected_persons,message A,low risk
1,123,connected_persons,message B,low risk
2,123,board_resolutions,message C,high risk
3,456,connected_persons,message A,low risk
4,789,connected_persons,message B,low risk
5,789,board_resolutions,message C,high risk


In [3]:
df_messages = df[["acctNo", "story", "message"]]
df_messages["risk"] = "warnings"
df_messages = df_messages.pivot_table(index = "acctNo", columns = ["story", "risk"], values = ["message"], aggfunc = lambda x: "\n\n".join(x))["message"]
df_messages

story,board_resolutions,connected_persons
risk,warnings,warnings
acctNo,,
123,message C,message A\n\nmessage B
456,NaN,message A
789,message C,message B


In [4]:
df_counts = pd.crosstab(index = df.acctNo, columns = [df.story, df.risk], dropna = False)
df_counts

story  board_resolutions          connected_persons         
risk           high risk low risk         high risk low risk
acctNo                                                      
123                    1        0                 0        2
456                    0        0                 0        1
789                    1        0                 0        1

In [5]:
out = df_counts.merge(df_messages, on = "acctNo")
out

story  board_resolutions          connected_persons           \
risk           high risk low risk         high risk low risk   
acctNo                                                         
123                    1        0                 0        2   
456                    0        0                 0        1   
789                    1        0                 0        1   

story  board_resolutions       connected_persons  
risk            warnings                warnings  
acctNo                                            
123            message C  message A\n\nmessage B  
456                  NaN               message A  
789            message C               message B

In [6]:
out = out[out.columns.sort_values()]
out

story  board_resolutions                     connected_persons           \
risk           high risk low risk   warnings         high risk low risk   
acctNo                                                                    
123                    1        0  message C                 0        2   
456                    0        0        NaN                 0        1   
789                    1        0  message C                 0        1   

story                           
risk                  warnings  
acctNo                          
123     message A\n\nmessage B  
456                  message A  
789                  message B

In [7]:
out.columns.names = ["", "acctNo"]
out

board_resolutions                     connected_persons           \
acctNo         high risk low risk   warnings         high risk low risk   
acctNo                                                                    
123                    1        0  message C                 0        2   
456                    0        0        NaN                 0        1   
789                    1        0  message C                 0        1   

                                
acctNo                warnings  
acctNo                          
123     message A\n\nmessage B  
456                  message A  
789                  message B

In [8]:
out.to_excel("test.xlsx")

In [9]:
import openpyxl

book = openpyxl.load_workbook("test.xlsx")
sheet = book["Sheet1"]
sheet.delete_rows(3, 1)    

In [10]:
book.save("test.xlsx")